import pandas as pd
import numpy as np

In [2]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install tensorflow
!pip install keras
!pip install xgboost
!pip install lightgbm
!pip install catboost
!pip install imblearn
!pip install sklearn
!pip install tqdm
!pip install requests pillow





[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



In [4]:
import os
import pandas as pd
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)


#1. DATA PREPARERATION

- Data loading and exploration
- Dataset statistics visualization
- Preprocessing steps
- Data splitting by skin types
- Initial data analysis

## RAW DATA

In [13]:
# Analysis of the Fitzpatrick 17k dataset

df = pd.read_csv('../data/csv/fitzpatrick17k.csv')

# Reduce the dataset to only include psoriasis samples
#df = df[df['label'] == 'psoriasis']

print("Columns in the dataset:", df.columns)
# print("Total samples:", len(df))
# print("\nFitzpatrick scale distribution:")
# print(df['fitzpatrick_scale'].value_counts().sort_index())
# print("\nNumber of unique conditions:", df['label'].nunique())
# print("\nTop 10 conditions:")
print("\nSample data:", df.head(10))
#print(df['label'].value_counts().head(10))
#print(['label']).unique()


Columns in the dataset: Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'three_partition_label', 'qc', 'url',
       'url_alphanum'],
      dtype='object')

Sample data:                             md5hash  fitzpatrick_scale  fitzpatrick_centaur  \
0  5e82a45bc5d78bd24ae9202d194423f8                  3                    3   
1  fa2911a9b13b6f8af79cb700937cc14f                  1                    1   
2  d2bac3c9e4499032ca8e9b07c7d3bc40                  2                    3   
3  0a94359e7eaacd7178e06b2823777789                  1                    1   
4  a39ec3b1f22c08a421fa20535e037bba                  1                    1   
5  45f7fe0e10214e32e890cad9d29d4811                  6                    5   
6  6c395be9325dbb10e55497304b398253                  2                    2   
7  9dc73230c77ab5c58dc1f11caef39ea2                  2                    1   
8  f23937e86de55c3471ac5d0143b67e08                  4           

## SET UP DATA

1. Utils: Function to create organized image storage structure for data

In [14]:
import os
import requests
from tqdm import tqdm
import hashlib

def setup_image_storage():
    # Create main image directory
    base_dir = 'data/images'
    os.makedirs(base_dir, exist_ok=True)
    
    # Create subdirectories for train/val/test (optional)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(base_dir, split), exist_ok=True)
    
    return base_dir

def download_image(url, filepath):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(response.content)
            return True
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")
        return False

def download_dataset_images(df, base_dir):
    """
    Download images from URLs and save them using md5hash as filename
    """
    print("Downloading images...")
    successful_downloads = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        # Create filepath using md5hash
        image_filename = f"{row['md5hash']}.jpg"
        filepath = os.path.join(base_dir, image_filename)
        
        # Skip if image already exists
        if os.path.exists(filepath):
            successful_downloads.append(filepath)
            continue
            
        # Download image if it doesn't exist
        if download_image(row['url'], filepath):
            successful_downloads.append(filepath)
    
    return successful_downloads

In [15]:
def update_df_with_image_paths(df, base_dir):
    """
    Add local image paths to the DataFrame
    """
    # Create image_path column
    df['image_path'] = df['md5hash'].apply(
        lambda x: os.path.join(base_dir, f"{x}.jpg")
    )
    
    # Verify images exist
    df['image_exists'] = df['image_path'].apply(os.path.exists)
    
    # Filter to only include existing images
    df_with_images = df[df['image_exists']].copy()
    df_with_images.drop('image_exists', axis=1, inplace=True)
    
    return df_with_images


3. Main execution pipeline

In [17]:
def prepare_dataset():
    # Load CSV
    df = pd.read_csv('../data/fitzpatrick17k.csv')
    
    # Setup image storage
    base_dir = setup_image_storage()
    
    # Download images
    successful_downloads = download_dataset_images(df, base_dir)
    print(f"Successfully downloaded {len(successful_downloads)} images")
    
    # Update DataFrame with image paths
    df_with_images = update_df_with_image_paths(df, base_dir)
    print(f"Total samples with images: {len(df_with_images)}")
    
    # Create train/val/test splits
    train_df, val_df, test_df = create_data_splits(df_with_images)
    
    return train_df, val_df, test_df

# Execute
if __name__ == "__main__":
    train_df, val_df, test_df = prepare_dataset()


  1%|          | 147/16577 [00:29<42:49,  6.39it/s]  

Error downloading nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


  2%|▏         | 351/16577 [01:10<38:24,  7.04it/s]  

Error downloading nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


  3%|▎         | 537/16577 [01:46<36:30,  7.32it/s]  

...

 76%|███████▌  | 12537/16577 [40:53<09:37,  6.99it/s]

Error downloading nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


100%|██████████| 16577/16577 [55:16<00:00,  5.00it/s]

Successfully downloaded 3887 images
Total samples with images: 3887


NameError: name 'create_data_splits' is not defined

In [21]:
from sklearn.model_selection import train_test_split
import shutil
import os
from tqdm import tqdm
import pandas as pd

def create_data_splits(df_with_images, base_dir='data/images', train_size=0.7, val_size=0.15):
    """
    Split data into train/val/test sets and organize images into respective folders
    
    Args:
        df_with_images: DataFrame with image paths
        base_dir: Base directory for images
        train_size: Proportion of data for training (default 0.7)
        val_size: Proportion of data for validation (default 0.15)
    Returns:
        train_df, val_df, test_df: DataFrames containing split data
    """
    print("Starting data split process...")
    
    # Create directories if they don't exist
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(base_dir, split)
        os.makedirs(split_dir, exist_ok=True)
        print(f"Created directory: {split_dir}")

    # First split into train and temp
    try:
        train_df, temp_df = train_test_split(
            df_with_images,
            train_size=train_size,
            stratify=df_with_images['fitzpatrick_scale'],
            random_state=42
        )
        
        # Split temp into val and test
        val_size_adjusted = val_size / (1 - train_size)
        val_df, test_df = train_test_split(
            temp_df,
            train_size=val_size_adjusted,
            stratify=temp_df['fitzpatrick_scale'],
            random_state=42
        )
        
        print("\nInitial split completed:")
        print(f"Training set: {len(train_df)} images")
        print(f"Validation set: {len(val_df)} images")
        print(f"Test set: {len(test_df)} images")
        
    except Exception as e:
        print(f"Error during data splitting: {str(e)}")
        return None, None, None

    # Function to move images
    def move_images(df, split_name):
        split_dir = os.path.join(base_dir, split_name)
        moved_paths = []
        
        print(f"\nMoving images to {split_name} folder...")
        for _, row in tqdm(df.iterrows(), total=len(df)):
            try:
                src_path = row['image_path']
                if os.path.exists(src_path):
                    # Create new path in split directory
                    dst_path = os.path.join(split_dir, f"{row['md5hash']}.jpg")
                    
                    # Copy the file if it doesn't exist in destination
                    if not os.path.exists(dst_path):
                        shutil.copy2(src_path, dst_path)
                    
                    moved_paths.append(dst_path)
                else:
                    print(f"Warning: Source image not found: {src_path}")
                    moved_paths.append(None)
            except Exception as e:
                print(f"Error moving image {row['md5hash']}: {str(e)}")
                moved_paths.append(None)
        
        return moved_paths

    # Move images and update paths
    print("\nOrganizing images into split folders...")
    train_df['image_path'] = move_images(train_df, 'train')
    val_df['image_path'] = move_images(val_df, 'val')
    test_df['image_path'] = move_images(test_df, 'test')

    # Remove rows with failed moves
    train_df = train_df.dropna(subset=['image_path'])
    val_df = val_df.dropna(subset=['image_path'])
    test_df = test_df.dropna(subset=['image_path'])

    # Print final statistics
    print("\nFinal dataset split sizes (after removing any failed transfers):")
    print(f"Training set: {len(train_df)} images")
    print(f"Validation set: {len(val_df)} images")
    print(f"Test set: {len(test_df)} images")

    # Save splits to CSV
    try:
        train_df.to_csv(os.path.join(base_dir, 'train_split.csv'), index=False)
        val_df.to_csv(os.path.join(base_dir, 'val_split.csv'), index=False)
        test_df.to_csv(os.path.join(base_dir, 'test_split.csv'), index=False)
        print("\nSplit CSVs saved successfully!")
    except Exception as e:
        print(f"Error saving CSV files: {str(e)}")

    return train_df, val_df, test_df

# Example usage:
if __name__ == "__main__":
    # Load your existing DataFrame
    try:
        base_dir = 'data/images'
        
        # Update image paths if needed
        df_with_images = update_df_with_image_paths(df, base_dir)
        
        # Create splits
        train_df, val_df, test_df = create_data_splits(df_with_images, base_dir)
        
        if train_df is not None:
            print("\nData split completed successfully!")
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")


Starting data split process...
Created directory: data/images/train
Created directory: data/images/val
Created directory: data/images/test

Initial split completed:
Training set: 2720 images
Validation set: 583 images
Test set: 584 images

Organizing images into split folders...

Moving images to train folder...


100%|██████████| 2720/2720 [00:01<00:00, 1872.71it/s]



Moving images to val folder...


100%|██████████| 583/583 [00:00<00:00, 1814.82it/s]



Moving images to test folder...


100%|██████████| 584/584 [00:00<00:00, 1747.47it/s]


Final dataset split sizes (after removing any failed transfers):
Training set: 2720 images
Validation set: 583 images
Test set: 584 images

Split CSVs saved successfully!

Data split completed successfully!


In [19]:
base_dir = 'data/images'

# Update image paths
df_with_images = update_df_with_image_paths(df, base_dir)

# Create splits and organize images
train_df, val_df, test_df = create_data_splits(df_with_images, base_dir)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

#2. BUILD MODEL

- nVGG-16 implementation
- Model architecture modifications
- Training setup
- Hyperparameter tuning

#3 Training_evaluation
- Model training
- Performance monitoring
- Learning curves
- Cross-validation results


#4 Results_analysis
- Performance metrics by skin type
- Visualization of results
- Bias analysis
- Comparison with original paper